<a href="https://colab.research.google.com/github/lucianosilvacraam/astro/blob/master/Astroinform%C3%A1tica_II_Aula_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASTROINFORMÁTICA II - Aula 09: Módulo simupy

http://github.com/simupy


In [0]:
!pip install simupy

In [0]:
from sympy.physics.mechanics import dynamicsymbols
from sympy.tensor.array import Array
from simupy.systems.symbolic import DynamicalSystem

x = x1, x2, x3 = Array(dynamicsymbols('x1:4'))
u = dynamicsymbols('u')
sys = DynamicalSystem(Array([-x1+x2-x3, -x1*x2-x2+u, -x1+u]), x, u)

In [0]:
from simupy.systems import LTISystem
ctrl = LTISystem([[1.73992128, 0.99212953,  -2.98819041]])

In [0]:
from simupy.block_diagram import BlockDiagram
BD = BlockDiagram(sys, ctrl)
BD.connect(sys, ctrl) # connect the current state to the feedback controller
BD.connect(ctrl, sys) # connect the controlled input to the system

In [0]:
sys.initial_condition = [5, -3, 1]
res = BD.simulate(10)